# Data Quality Audit & Cleaning Strategy

### **Objective**
To profile the raw transactional data and create a 'trusted analytics view' that preserves Total Revenue integrity while enabling granular customer segmentation

---
### **Methodology**
We performed the following integrity checks on the `raw_retail_transactions` table:
* *Completeness:* Checked for NULL values in key columns e.g. `customer_id`
* *Validity:* Verified logical consistency (e.g. flagged negative prices as "Bad Debt" and negative quantities as "Returns")
* *Uniqueness:* Checked for duplicate records to prevent double-counting
* *Outlier Detection:* Inspected extreme values in `Quantity` to identify anomalies that could skew averages
* *Timeline Verification:* Assessed the date range to ensure we are analyzing complete business periods

---
### **Key Decisions & Logic**

**Handling Null Customer IDs**
* **Finding:** A significant portion of transactions have no `customer_id`
* **Decision:** **RETAIN**
* **Reasoning:** These are valid "Guest Checkouts." Excluding them would artificially deflate Total Revenue and skew Geographic Sales analysis

**Handling Cancellations (Invoices starting with 'C')**
* **Finding:** Records starting with 'C' correspond to negative quantities
* **Decision:** **RETAIN (as Negatives)**
* **Reasoning:** 'C' indicates a Credit Note/Refund/Return. Keeping them allows us to calculate **Net Revenue** (Sales minus Returns) accurately

**Handling Bad Debt (StockCode 'B')**
* **Finding:** Negative prices associated with "Adjust Bad Debt"
* **Decision:** **EXCLUDE**
* **Reasoning:** These are internal accounting adjustments, not customer activity

**Handling Missing Descriptions**
* **Finding:** Records with NULL descriptions consistently have `Price = 0`
* **Decision:** **EXCLUDE**
* **Reasoning:** These represent system errors. Since `Total_Sales = 0`, they add no value and create noise

**Handling Date Range (2009 Data)**
* **Finding:** The dataset includes partial data from December 2009
* **Decision:** **EXCLUDE**
* **Reasoning:** To prevent seasonality bias, we will focus the analysis on the complete calendar year(s) starting from Jan 2010. Including a partial month from 2009 would skew "Q4" or "December" comparisons.

---
### **Final Output**
We will create a SQL View `online_retail_clean` that cleans the data using `DISTINCT` to handle duplicates, while applying the filters above to **preserve the full financial picture.**

In [1]:
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv; load_dotenv() #noqa

db_connection_str = os.getenv('DB_CONNECTION_STR')
db_connection = create_engine(db_connection_str)

In [15]:
%load_ext sql
%config SqlMagic.displaycon = False  # disables the conn string display    # disables the "Done" and row count
%config SqlMagic.feedback = False
%sql {db_connection_str}

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [21]:
# show sample data
%sql select * from raw_retail_transactions limit 3;

invoice,stock_code,description,quantity,invoice_date,price,customer_id,country
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085,United Kingdom


In [81]:
%%sql
-- check duplicate records
SELECT
    invoice,
    stock_code,
    description,
    quantity,
    invoice_date,
    price,
    customer_id,
    country,
    count(*) records_count
FROM raw_retail_transactions
GROUP BY
    invoice,
    stock_code,
    description,
    quantity,
    invoice_date,
    price,
    customer_id,
    country
HAVING count(*) > 1
limit 3
;

invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,records_count
489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01,1.95,16329,United Kingdom,2
489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01,3.75,16329,United Kingdom,2
489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01,3.75,16329,United Kingdom,3


In [ ]:
%%sql
-- checking the percentage of missing customer_id records
with cte as (
    select
        count(*) as total_records,
        count(customer_id) as records_with_customer_id,
        count(*) - count(customer_id) as records_missing_customer_id
    from raw_retail_transactions
    )
select
    total_records,
    records_with_customer_id,
    records_missing_customer_id,
    round((records_missing_customer_id::decimal / total_records) * 100) || '%' as percentage_missing_customer_id
from cte
;

total_records,records_with_customer_id,records_missing_customer_id,percentage_missing_customer_id
1067371,824364,243007,23%


In [27]:
%%sql
-- checking yearly transactions and total transactions (total records in tb)
-- this helps to understand if we have a full year data for analysis
with cte as (
    select
        extract(year from invoice_date) as invoice_year,
        min(invoice_date) as earliest_invoice_date,
        max(invoice_date) as latest_invoice_date,
        count(*) as yearly_transactions
    from raw_retail_transactions
    group by invoice_year
    order by invoice_year
    )
select
    *,
    sum(yearly_transactions) over () as total_transactions
from cte
;

invoice_year,earliest_invoice_date,latest_invoice_date,yearly_transactions,total_transactions
2009,2009-12-01,2009-12-23,45228,1067371
2010,2010-01-04,2010-12-23,522714,1067371
2011,2011-01-04,2011-12-09,499429,1067371


In [66]:
%%sql
-- sense check description field for any null or empty values
select
    count(*) as total_missing_descriptions,
    sum(quantity * price) as total_sales
from raw_retail_transactions
where description is null
;

total_missing_descriptions,total_sales
4382,0.00


In [34]:
%%sql
-- check price & quantity ranges
select
    min(price) as min_price,
    max(price) as max_price,
    min(quantity) as min_quantity,
    max(quantity) as max_quantity
from raw_retail_transactions
;

min_price,max_price,min_quantity,max_quantity
-53594.36,38970.00,-80995,80995


In [31]:
%%sql
-- check records with negative price
select
    *
from raw_retail_transactions
where price = (select min(price) from raw_retail_transactions)
    or price < 0
;

invoice,stock_code,description,quantity,invoice_date,price,customer_id,country
A516228,B,Adjust bad debt,1,2010-07-19,-44031.79,None,United Kingdom
A506401,B,Adjust bad debt,1,2010-04-29,-53594.36,None,United Kingdom
A528059,B,Adjust bad debt,1,2010-10-20,-38925.87,None,United Kingdom
A563186,B,Adjust bad debt,1,2011-08-12,-11062.06,None,United Kingdom
A563187,B,Adjust bad debt,1,2011-08-12,-11062.06,None,United Kingdom


In [29]:
%%sql
-- checking records with stock_code 'B'
select
    *
from raw_retail_transactions
where stock_code = 'B'
;

invoice,stock_code,description,quantity,invoice_date,price,customer_id,country
A506401,B,Adjust bad debt,1,2010-04-29,-53594.36,None,United Kingdom
A528059,B,Adjust bad debt,1,2010-10-20,-38925.87,None,United Kingdom
A563185,B,Adjust bad debt,1,2011-08-12,11062.06,None,United Kingdom
A563186,B,Adjust bad debt,1,2011-08-12,-11062.06,None,United Kingdom
A563187,B,Adjust bad debt,1,2011-08-12,-11062.06,None,United Kingdom
A516228,B,Adjust bad debt,1,2010-07-19,-44031.79,None,United Kingdom


In [54]:
%%sql
-- checking records with invoice starts with 'A'
select
    *
from raw_retail_transactions
where invoice ilike 'A%'
;

invoice,stock_code,description,quantity,invoice_date,price,customer_id,country
A506401,B,Adjust bad debt,1,2010-04-29,-53594.36,None,United Kingdom
A516228,B,Adjust bad debt,1,2010-07-19,-44031.79,None,United Kingdom
A563185,B,Adjust bad debt,1,2011-08-12,11062.06,None,United Kingdom
A563186,B,Adjust bad debt,1,2011-08-12,-11062.06,None,United Kingdom
A563187,B,Adjust bad debt,1,2011-08-12,-11062.06,None,United Kingdom
A528059,B,Adjust bad debt,1,2010-10-20,-38925.87,None,United Kingdom


In [43]:
%%sql
-- check records with negative quantity
select
    *
from raw_retail_transactions
where quantity = (select max(quantity) from raw_retail_transactions)
    or quantity = (select min(quantity) from raw_retail_transactions)
;

invoice,stock_code,description,quantity,invoice_date,price,customer_id,country
581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09,2.08,16446,United Kingdom
C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09,2.08,16446,United Kingdom


In [48]:
%%sql
-- check invoice start with 'C'
select
    *
from raw_retail_transactions
where invoice ilike 'C%'
    and quantity >= 0
;

invoice,stock_code,description,quantity,invoice_date,price,customer_id,country
C496350,M,Manual,1,2010-02-01,373.57,None,United Kingdom


In [52]:
%%sql
-- check invoice start with 'C'
select
    count(*) as total_credit_invoices
from raw_retail_transactions
where invoice ilike 'C%'
;

total_credit_invoices
19494


#### create SQL View `online_retail_clean` 

In [82]:
%%sql
DROP VIEW IF EXISTS online_retail_clean;

CREATE VIEW online_retail_clean AS
SELECT distinct
    country,
    customer_id,
    invoice_date,
    invoice,
    stock_code,
    description,
    quantity,
    price,
    (quantity * price) as invoice_amount
FROM raw_retail_transactions
WHERE
    -- exclude accounting adjustment records
    stock_code != 'B'
    -- exclude records with NULL description
    AND description IS NOT NULL
    -- exclude 2009 data
    AND extract(year from invoice_date) != 2009

[]

In [83]:
%sql select count(*) from online_retail_clean;

count
984260
